# COSA Flood Exploration

## Imports

In [1]:
import wrangle as w
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#Fix some errors
# Flood Cleaning
def clean_flood():
    '''Drops unneeded columns from the med center flooding df
    Makes sure DateTime is in DateTime format'''
    # read csv
    flood = pd.read_csv('med_center_flood.csv')
    # drop the columns
    flood = flood.drop(columns=['LAT', 'LONG', 'Zone',  
                          'SensorStatus', 'AlertTriggered', 
                          'Temp_C', 'Temp_F', 'Vendor'])
    # Set to date time format
    flood.DateTime = pd.to_datetime(flood.DateTime)
    flood = flood.rename(columns={"DateTime": "datetime", 
                        "DistToWL_ft": "sensor_to_water_feet", 
                        "DistToWL_m": "sensor_to_water_meters", 
                        "DistToDF_ft": "sensor_to_ground_feet",
                        "DistToDF_m": "sensor_to_ground_meters"})
    # replae -999 with 0
    flood["sensor_to_ground_feet"].replace({-999:13.5006561680}, inplace=True)
    flood["sensor_to_ground_meters"].replace({-999:4.115}, inplace=True)
    
    #flood = flood.replace(to_replace=-999, value=0)
    # create new features for flood depth
    flood['flood_depth_feet'] = flood.sensor_to_ground_feet - flood.sensor_to_water_feet
    flood['flood_depth_meters'] = flood.sensor_to_ground_meters - flood.sensor_to_water_meters 
    # Create new alert
    def flood_alert(c):
        if 0 < c['flood_depth_feet'] < 10:
            return 'No Risk'
        elif 10 < c['flood_depth_feet'] < 11:
            return 'Minor Risk'
        elif 11 < c['flood_depth_feet'] < 12:
            return 'Moderate Risk'
        elif 12 < c['flood_depth_feet']:
            return 'Major Risk !'
        else:
            return 'No Alert'
    flood['flood_alert'] = flood.apply(flood_alert, axis=1)
    flood = flood[(flood.sensor_to_water_feet != -999)]
    # return new df
    return flood

In [3]:
flood = clean_flood()

In [4]:
flood.head()

,datetime,Sensor_id,SensorModel,sensor_to_water_feet,sensor_to_water_meters,sensor_to_ground_feet,sensor_to_ground_meters,flood_depth_feet,flood_depth_meters,flood_alert
0,2021-05-20 00:11:55,400789,ranger sensor,13.188976,4.02,13.500656,4.115,0.311680,0.095,No Risk
1,2021-05-20 00:26:55,400789,ranger sensor,13.254593,4.04,13.500656,4.115,0.246063,0.075,No Risk
2,2021-05-20 00:41:55,400789,ranger sensor,13.221785,4.03,13.500656,4.115,0.278871,0.085,No Risk
3,2021-05-20 00:56:55,400789,ranger sensor,13.221785,4.03,13.500656,4.115,0.278871,0.085,No Risk
4,2021-05-20 01:11:55,400789,ranger sensor,13.221785,4.03,13.500656,4.115,0.278871,0.085,No Risk


In [ ]:
#flood.to_csv('bestflood.csv')

In [5]:
flood.sensor_to_ground_feet.value_counts()

13.500656    17834
13.500656     7559
Name: sensor_to_ground_feet, dtype: int64

In [6]:
flood.describe()

,Sensor_id,sensor_to_water_feet,sensor_to_water_meters,sensor_to_ground_feet,sensor_to_ground_meters,flood_depth_feet,flood_depth_meters
count,25393.000000,25393.000000,25393.000000,2.539300e+04,2.539300e+04,25393.000000,25393.000000
mean,162462.483716,4.083000,1.244497,1.350066e+01,4.115000e+00,9.417657,2.870503
std,155163.192945,5.800390,1.767960,9.821836e-12,1.535691e-12,5.800390,1.767960
min,61447.000000,0.000000,0.000000,1.350066e+01,4.115000e+00,0.016404,0.005000
25%,61447.000000,0.233300,0.071100,1.350066e+01,4.115000e+00,0.606955,0.185000
50%,61447.000000,0.291700,0.088900,1.350066e+01,4.115000e+00,13.208956,4.026100
75%,400789.000000,12.893701,3.930000,1.350066e+01,4.115000e+00,13.267356,4.043900
max,400789.000000,13.484252,4.110000,1.350066e+01,4.115000e+00,13.500656,4.115000


In [7]:
# Shape before dropping -999
flood.shape

(25393, 10)

In [ ]:
flood.flood_depth_feet

In [ ]:
# drop the -999
flood2 = flood[(flood.sensor_to_water_feet != -999)]

In [ ]:
# shape after -999 drop
flood2.shape

In [ ]:
#descriptive statistics
flood2.describe()

### Tableau Link